In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/data_and_models

/content/gdrive/MyDrive/data_and_models


In [20]:
import functools
import os
from os import listdir
import json
import pandas as pd
import keras

In [37]:
def evaluate(name_model, speciphic_name , data):
  metadata =  f"{data}/data/{name_model}/metainfo.json"
  data_path = f"{data}/data/{name_model}/{speciphic_name}"
  model_path_ = f"{data}/models/{name_model}/{speciphic_name}"
  model_path = f"{model_path_}/{os.listdir(model_path_)[0]}"
  with open(metadata, "r") as read_file:
    js_file = json.load(read_file)
  inp = js_file[speciphic_name]["n_samples"]
  inp_shape = js_file[speciphic_name]["input_dim"]
  output_dim = js_file[speciphic_name]["output_dim"]
  inp_shape_flatten = functools.reduce(lambda a, b: a*b, inp_shape)

  print("getting data")
  df = pd.read_csv (data_path + '/train.csv').to_numpy()

  X_train = df[:,:inp_shape_flatten].reshape(-1,*inp_shape)
  y_train = df[:,inp_shape_flatten:].reshape(-1,*output_dim)
  print("loading model")
  model = keras.models.load_model(model_path)
  print(model.summary())
  model.evaluate(X_train,y_train,batch_size=100)

In [11]:
datas={}
path = "/content/gdrive/MyDrive/data_and_models/data"
for p in os.listdir(path):
  sub_dir_p = list(filter(os.path.isdir,[path +"/"+ p +"/"+ pa for pa in os.listdir(path+"/"+p)]))
  datas[p] = list(map(lambda x : x.split("/")[-1],sub_dir_p))

In [38]:
for key, value in datas.items():
  if key == "channel_decoding": continue
  for v in value:
    print(f"\n\n\t\t\t {key}     {v}")
    evaluate(key,v,"/content/gdrive/MyDrive/data_and_models")



				 beam_selection     input=beam_subset
getting data
loading model
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 2, 16, 32)]       0         
                                                                 
 tf.math.subtract_7 (TFOpLam  (None, 2, 16, 32)        0         
 bda)                                                            
                                                                 
 tf.math.truediv_7 (TFOpLamb  (None, 2, 16, 32)        0         
 da)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 2048)              2099200   
                                                      

## channel decoding model seems to have trouble to load metrics  

In [63]:
model_path = "/content/gdrive/MyDrive/data_and_models/models/channel_decoding/coding_scheme=convolutional/a6c2c4e6-d13e-37c6-a5a6-f625e69bfc82"
model = keras.models.load_model(model_path)

ValueError: ignored

so i just tried to check the accuracy by defining metric after loading model 

I used bit error rate that was defined in utils 

In [60]:
from tensorflow.keras.metrics import MeanMetricWrapper
from tensorflow import cast, abs, not_equal, round
def bit_error_rate(y_true, y_pred):
    return abs(cast(not_equal(y_true, round(y_pred)), dtype='float32'))

In [64]:
def evaluate1(name_model, speciphic_name , data):
  metadata =  f"{data}/data/{name_model}/metainfo.json"
  data_path = f"{data}/data/{name_model}/{speciphic_name}"
  model_path_ = f"{data}/models/{name_model}/{speciphic_name}"
  model_path = f"{model_path_}/{os.listdir(model_path_)[0]}"
  with open(metadata, "r") as read_file:
    js_file = json.load(read_file)
  inp = js_file[speciphic_name]["n_samples"]
  inp_shape = js_file[speciphic_name]["input_dim"]
  output_dim = js_file[speciphic_name]["output_dim"]
  inp_shape_flatten = functools.reduce(lambda a, b: a*b, inp_shape)

  print("getting data")
  df = pd.read_csv (data_path + '/train.csv').to_numpy()

  X_train = df[:,:inp_shape_flatten].reshape(-1,*inp_shape)
  y_train = df[:,inp_shape_flatten:].reshape(-1,*output_dim)
  print("loading model")
  model = keras.models.load_model(model_path,compile =False )
  model.compile( loss=None,metrics=bit_error_rate)
  print(model.summary())
  model.evaluate(X_train,y_train,batch_size=100)

In [66]:
for key, value in datas.items():
  if key == "channel_decoding": 
      for v in value:
        print(f"\n\n\t\t\t {key}     {v}")
        evaluate1(key,v,"/content/gdrive/MyDrive/data_and_models")



			 channel_decoding     coding_scheme=convolutional
getting data
loading model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 1)]          0         
                                                                 
 tf.math.subtract (TFOpLambd  (None, 100, 1)           0         
 a)                                                              
                                                                 
 tf.math.truediv (TFOpLambda  (None, 100, 1)           0         
 )                                                               
                                                                 
 reshape (Reshape)           (None, 100, 1)            0         
                                                                 
 lstm (LSTM)                 (None, 256)               264192    
                                             